# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [142]:
# Write your code below.
%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [143]:
#import dask
#dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd
import numpy as np
import warnings # for muting warning messages
# mute warning messages
warnings.filterwarnings('ignore')

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [144]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
price_files = glob(PRICE_DATA+"/**/*.parquet/*.parquet")
price_df = dd.read_parquet(price_files)

For each ticker and using Dask, do the following:

+ Add lags for variable Close
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [146]:
# Write your code below.
# Calculate lagged values and returns
dd_feat = (price_df.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        hi_lo_range = x['High'] - x['Low']
    )
)
.assign(
    returns = lambda x: (x['Close'] / x['Close_lag_1']) - 1
))

dd_feat = dd_feat.persist()

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [147]:
# Write your code below.

# Convert the Dask data frame to a pandas data frame. 
dd_feat = dd_feat.compute()

# Add a rolling average return calculation with a window of 10 days.
grouped_data = dd_feat.groupby('ticker')

def calculate_rolling_return(group):
  min_window = min(10, len(group))
  group['rolling_return_10d'] = group['returns'].rolling(min_window).mean()
  return group

dd_feat = grouped_data.apply(calculate_rolling_return).reset_index(level=1, drop=True)

```Please comment:```

#### + Was it necessary to convert to pandas to calculate the moving average return?
````
No, it was not necessary. Dask DataFrames support many pandas operations, including calculating moving averages, directly on Dask objects. This means you can compute moving averages efficiently using Dask without needing to convert your data to a pandas DataFrame first.
````
#### + Would it have been better to do it in Dask? 
```
Yes, it would have been better to calculate the moving average return in Dask, especially for handling large datasets such as stock market data.
```
#### Why?
```
1. Dask support many pandas operations, including computing moving averages
2. Dask leverages parallel computing, optimizing performance which allows to handle datasets that are larger-than-memory calculating moving averages efficiently
```

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.